In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [ ]:
%cd gdrive/My Drive/

In [ ]:
%pwd
%mkdir proyecto

In [ ]:

%cd ./proyecto/

In [ ]:
%pwd

In [ ]:
!pip install pyts

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyts.decomposition import SingularSpectrumAnalysis
import scipy.io as sc
from scipy.signal import freqz

In [ ]:
%pwd


In [ ]:
%ls

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)    
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y
def butter_lowpass_bode (b ,a):
    angularFreq, freqResponse, phaseResponse = scipy.signal.bode(scipy.signal.lti(b, a))

    return angularFreq, freqResponse, phaseResponse


In [ ]:
#Tele
data_tele = sc.loadmat('/content/gdrive/My Drive/proyecto/sisTele.mat')
data_tele.keys() # variables del diccionario

Y_tele = data_tele['Y']
fs_tele = data_tele['fs']
fs_tele = fs_tele[0][0]
print("muestreo",fs_tele)


plt.plot(Y_tele.T) #se le hace transpuesta
plt.xlabel('sample')
plt.ylabel('$\mu$ V')
plt.show()
sampleSignal = Y_tele[3,:] # se extrae una señal de la base de datos
plt.figure
plt.plot(sampleSignal)
yy = sampleSignal
plt.xlabel('sample')
plt.ylabel('$\mu$ V')
plt.show()

print(sampleSignal[:80])
#regional 

data_reg = sc.loadmat('/content/gdrive/My Drive/proyecto/sisReg.mat')
data_reg.keys() # variables del diccionario

Y_reg = data_reg['Y']
fs_reg = data_reg['fs']

#Local


data_loc = sc.loadmat('/content/gdrive/My Drive/proyecto/sisLoc.mat')
data_loc.keys() # variables del diccionario

Y_loc = data_loc['Y']
fs_loc = data_loc['fs']

In [ ]:
fs = fs_tele + 1
lowcut = 1.0
highcut = 50
cutOff = 50

# Plot the frequency response for a few different orders.
plt.figure(1)
plt.clf()
for order in [3, 6, 9]:
  b, a = butter_lowpass(cutOff, fs, order)
  w, h = freqz(b, a, worN=2000)
  plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)

plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
             '--', label='sqrt(0.5)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain')
plt.grid(True)
plt.legend(loc='best')

plt.figure(2)
plt.clf()
plt.plot(yy, label='Noisy signal')


y = butter_lowpass_filter(yy,cutOff, fs, order=3)
plt.plot(y, label='Filtered signal (%g Hz)' )
plt.xlabel('samples')
plt.grid(True)
plt.axis('tight')
plt.legend(loc='lower right')

plt.show()

In [ ]:
zeos = np.zeros((2,3))
print(zeos)
zeos[0] = [1,2,3]
print(zeos)

In [ ]:
Y_tele.shape[0]


In [ ]:
x = len(y)-len(yy)
a = len(y) - x
y = y[:32000]


Aqui hare el filtro a las señales del tele sismo

In [ ]:
signalFiltered = np.zeros((Y_tele.shape[0],Y_tele.shape[1]))
fs = fs_tele + 1

cutOff = 50

order = 6

'''b, a = butter_lowpass(cutOff, fs, order)
angularFreq, freqResponse, phaseResponse = scipy.signal.bode(scipy.signal.lti(b, a))
plt.figure(i)
plt.semilogx(angularFreq, freqResponse)
plt.grid(True, which = "both")
plt.title(i)
plt.figure(i+1)
plt.semilogx(angularFreq, phaseResponse)
plt.grid(True, which = "both")'''
for i in [3,6,9]:
  b, a = butter_lowpass(cutOff, fs, order=i)
  angularFreq, freqResponse, phaseResponse = scipy.signal.bode(scipy.signal.lti(b, a))
  plt.figure(i)
  plt.semilogx(angularFreq, freqResponse)
  plt.grid(True, which = "both")
  plt.title(i)
  plt.figure(i+1)
  plt.semilogx(angularFreq, phaseResponse)
  plt.grid(True, which = "both")

  plt.figure(figsize=(10,5))
  
for j in range (0,Y_tele.shape[0]):
  signalFiltered[j,:] = butter_lowpass_filter(Y_tele[j,:], cutOff, fs, order=6)
  print(j)
  print(signalFiltered[j,:10])